# 4-7. 프로젝트: 멋진 작사가 만들기

---

## Step 1. 데이터 다운로드

[Song Lyrics](https://www.kaggle.com/paultimothymooney/poetry/data)에서 다운받을 수 있다.

## Step 2. 데이터 읽어오기

In [1]:
import glob
import os
import re 
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# 필요한 모듈 불러오기

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:5])

데이터 크기: 187088
Examples:
 ['[Verse 1]', 'They come from everywhere', 'A longing to be free', 'They come to join us here', 'From sea to shining sea And they all have a dream']


## Step 3. 데이터 정제

In [3]:
# 정규표현식을 통한 문장정리
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue  # 길이가 0인 빈 문장은 건너뜁니다
    if sentence[-1] == "]": continue # ']'로 끝나는 파트를 나눈 문장은 건너뜁니다
    if sentence[-1] == ")": continue # ')'로 끝나는 코러스 문장은 건너뜁니다    
        
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> they come from everywhere <end>',
 '<start> a longing to be free <end>',
 '<start> they come to join us here <end>',
 '<start> from sea to shining sea and they all have a dream <end>',
 '<start> as people always will <end>',
 '<start> to be safe and warm <end>',
 '<start> in that shining city on the hill some wanna slam the door <end>',
 '<start> instead of opening the gate <end>',
 '<start> aw , let s turn this thing around <end>',
 '<start> it s up to me and you <end>']

In [5]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    # maxlen=15로 설정
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   42   71 ...    0    0    0]
 [   2    9 3391 ...    0    0    0]
 [   2   42   71 ...    0    0    0]
 ...
 [   2  557   20 ...    0    0    0]
 [   2  121   33 ...    0    0    0]
 [   2    5   22 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f1ecf7325d0>


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])  # 단어사전 내용 확인

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[  2  42  71  72 790   3   0   0   0   0   0   0   0   0]
[ 42  71  72 790   3   0   0   0   0   0   0   0   0   0]


## Step 4. 평가 데이터셋 분리

In [8]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (133342, 14)
Target Train: (133342, 14)


In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

## Step 5. 인공지능 만들기

In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 4096
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [11]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-1.6259715e-04,  1.5333512e-04,  5.8808125e-04, ...,
          3.1664100e-04,  4.6281271e-05,  1.3006509e-04],
        [-2.7198318e-04,  2.5143757e-04,  8.9717534e-04, ...,
          3.6795135e-04, -1.7344527e-04, -1.5385922e-04],
        [-7.4319847e-05,  5.3340843e-04,  1.5906260e-03, ...,
          8.5882784e-05, -4.0709937e-04, -3.8016125e-04],
        ...,
        [-2.1036714e-03, -3.2223170e-03,  3.2836576e-03, ...,
         -2.5081239e-03, -9.0050098e-04, -2.7710190e-05],
        [-2.4442265e-03, -3.7757859e-03,  3.6823251e-03, ...,
         -3.0737608e-03, -1.3545586e-03, -3.8213006e-04],
        [-2.7399117e-03, -4.2523094e-03,  4.0362934e-03, ...,
         -3.5687408e-03, -1.8201264e-03, -7.1501598e-04]],

       [[-1.6259715e-04,  1.5333512e-04,  5.8808125e-04, ...,
          3.1664100e-04,  4.6281271e-05,  1.3006509e-04],
        [ 3.1244615e-04,  1.5779736e-04,  3.2344091e-04, ...,
          6.6679151e-04, 

In [12]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  12289024  
_________________________________________________________________
lstm (LSTM)                  multiple                  83902464  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  134234112 
_________________________________________________________________
dense (Dense)                multiple                  49168097  
Total params: 279,593,697
Trainable params: 279,593,697
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, validation_data=(enc_val, dec_val),epochs=10, batch_size=512)

Epoch 1/10
261/261 [==============================] - 1570s 6s/step - loss: 4.0624 - val_loss: 3.2469
Epoch 2/10
261/261 [==============================] - 1586s 6s/step - loss: 3.1169 - val_loss: 2.9541
Epoch 3/10
261/261 [==============================] - 1586s 6s/step - loss: 2.7691 - val_loss: 2.7502
Epoch 4/10
261/261 [==============================] - 1581s 6s/step - loss: 2.4331 - val_loss: 2.5853
Epoch 5/10
261/261 [==============================] - 1580s 6s/step - loss: 2.0898 - val_loss: 2.4556
Epoch 6/10
261/261 [==============================] - 1583s 6s/step - loss: 1.7472 - val_loss: 2.3547
Epoch 7/10
261/261 [==============================] - 1578s 6s/step - loss: 1.4421 - val_loss: 2.2909
Epoch 8/10
261/261 [==============================] - 1576s 6s/step - loss: 1.2015 - val_loss: 2.2704
Epoch 9/10
261/261 [==============================] - 1581s 6s/step - loss: 1.0366 - val_loss: 2.2816
Epoch 10/10
261/261 [==============================] - 1581s 6s/step - loss: 0.947

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you more than i did <end> '

---

## 평가 관련 고려한 점

**1. 가사 텍스트 생성 모델이 정상적으로 동작하는가?**

- 확인해보면 문맥에 맞는 문장을 구사함을 알 수 있다.
      i love -> i love you more than i did
- 학습을 잘못시켜서 동일한 단어만 반복하는 케이스도 발생했는데 소스와 타겟문장 생성에 주의해야함을 깨달았다.


**2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?**

- 전처리: 데이터로 주어진 가사집들을 확인해보며 삭제해야 할 부분에 대해 고민했다.
- \[Verse 1\]로 되어 있는 부분은 전체 줄을 삭제했으나 가사와 함께 표시되어 있는 \[\]부분에 대해 고민했다. -> ]으로 종료되는 문장 삭제
- ()안에 들어 있는 코러스에 대해서 가사가 있어 문장 구성에 도움이 되나 단순 효과음인 구절도 있는 부분에 대해서는 어떻게 처리할지 고민했다. -> 학습에 방해가 될 것을 고려하여 삭제
- 노래 제목(nursery_rhymes.txt)도 함께 표기된 표기된 경우 제목 또한 문장으로서의 역할을 하기에 살렸다.
- 토크나이저 생성의 경우 노드에 나온 설명대로 토큰 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권하여  maxlen을 15 설정하였다.
- 데이터셋 같은 경우 훈련과 평가 데이터를 나누는데 8:2가 보편적이다 하여 test_size는 0.2로 설정하였다.


**3. 텍스트 생성모델이 안정적으로 학습되었는가?**

- 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하였다.
- num_words, batch size, embedding size, hidden size 등 수치 변화를 통해 validation loss를 맞추려고 여러차례 반복했다.
- 직전에는 다른 변수들은 고정해놓고 embedding size, hidden size 조정을 통해 loss값을 맞춰봤다.
- Total params가 늘어날 수록 학습시간이 오래걸렸다. 그렇다고 해서 loss 값이 목표치 만큼 나오지는 않았다.
- hidden_size를 4096까지 늘려 Total params가 약 2억8천이 되었다. hidden_size가 2048일때 val_loss값 2.3568이였던 것을 생각해보면 2048과 4096의 중간값정도로 조정하는 게 학습시간 효율성 측면에서 더 나을 듯 싶다.

## 회고

- 1차 시도때 loss값이 2.2566가 나와서 조금만 수정하면 된다고 생각했던 것이 오산이였다.
- 모델이 정상적으로 작동하는 것과 loss값을 맞추는 것 중 모델의 정상 작동이 더 중요하다고 느꼈다. -> 잘못된 학습으로 동일 단어 반복할때의 loss값은 0.9이하였다.
- 루브릭 평가 기준인 validation loss가 2.2 이하로 맞추는 것만 신경쓰며 수정한 것 같다.    
- 생각해보면 올바른 모델이란 loss값보다도 모델 목적에 맞게 정확한 문장을 구하는 모델이 아닐까? 
  -> loss값이 2.2이하로 떨어지지 않았던 경우에도 'i love you , i m a fool'와 같은 멋진 가사가 나왔다.
  
  
- 하이퍼파라미터 설정에 대해 시도해보고 고민할 수 있었던 EXPLORATION이였다.
- 모델에 비해 학습시간이 많이 걸렸다는 생각이 든다. 좀더 확실하게 정확도를 내고 싶어서 설정했지만 좋은 방법인지에 대한 고민이 필요할 것 같다.
- 훈련 시간을 줄이면서 좋은 모델을 만드는 방법에 대해 알고 싶다.

- 9 에폭부터 val_loss값이 다시 증가하는 모습이 보였다. 지금은 제출기한 정해져있어 제출하나, 해당 부분에 대한 검토가 더 필요할 것 같다.

- 적진 않겠지만 모델이 생성한 가사가 자칫하면 부적절하게 해석될만한 가사가 나오기도 했다.
- 방송 심의에 통과할 수 있는 가사를 만드는 모델을 만들려면 어떤 부분을 수정해야하는 지 더 공부하고 싶다.